#### IMPORTANT: Kernel used on HiPerGator - Yolo-v8

In [10]:
# Install torchmetrics.
!pip install torchmetrics

In [2]:
# Import necessary libraries
import os
import torch
import numpy as np
from pprint import pprint
from ultralytics import YOLO
from torchmetrics.detection import IntersectionOverUnion
from torchmetrics.detection import MeanAveragePrecision

In [3]:
# Give the path to 'best.pt' file which is inside the weights folder. The folder has been uploaded into git repo
# Find the example path below
model = YOLO('runs/detect/train10/weights/best.pt')

# Specify the path of the folder which contain test images '.jpg' files
test_images = '../datasets/valid/images'

# Specify the path of the folder which contains test labels '.txt' files
test_labels = "../datasets/valid/labels"

In [4]:
def predict_objects(model, test_images):
    """
    This function predict and save the predicted coordiantes, class labels and confidence score
    into .txt files, one per image with the same file name.
    """
    results_detection = model.predict(source = test_images, save_txt= True, save_conf = True)
    result_dir = results_detection[0].save_dir
    pred_path = str(result_dir)
    return pred_path

In [8]:
# We call the predict function to run the prediction on test set
# and store the path to the prediction labels into a variable
pred_path = predict_objects(model, test_images)

In [6]:
# This code cell performs preparation of data for iou and map calculations

def yolo_to_abs_coords(yolo_data, img_width, img_height, include_scores=False):
    """
    Convert YOLO format to (xmin, ymin, xmax, ymax) with optional scores.
    """
    abs_boxes = []
    labels = []
    scores = []
    
    for entry in yolo_data:
        if include_scores:
            cls, x_center, y_center, width, height, score = entry
            scores.append(float(score))
        else:
            cls, x_center, y_center, width, height = entry
        
        xmin = (x_center - width / 2) * img_width
        ymin = (y_center - height / 2) * img_height
        xmax = (x_center + width / 2) * img_width
        ymax = (y_center + height / 2) * img_height
        
        abs_boxes.append([xmin, ymin, xmax, ymax])
        labels.append(int(cls))  # Convert class to integer
    
    if include_scores:
        return (torch.tensor(abs_boxes, dtype=torch.float32), 
                torch.tensor(labels, dtype=torch.int64),
                torch.tensor(scores, dtype=torch.float32))
    else:
        return torch.tensor(abs_boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

def parse_yolo_file(file_path, include_scores=False):
    """
    Parse a YOLO .txt file into a list of tuples. Includes scores if specified.
    """
    with open(file_path, "r") as f:
        lines = f.readlines()
    return [list(map(float, line.strip().split())) for line in lines]

def prepare_data(file_paths, img_width, img_height, include_scores=False):
    """
    Prepare data for multiple YOLO files into the required format for PyTorch metrics.
    """
    data = []
    for file_path in file_paths:
        yolo_data = parse_yolo_file(file_path, include_scores)
        if include_scores:
            boxes, labels, scores = yolo_to_abs_coords(yolo_data, img_width, img_height, include_scores)
            data.append({"boxes": boxes, "labels": labels, "scores": scores})
        else:
            boxes, labels = yolo_to_abs_coords(yolo_data, img_width, img_height, include_scores)
            data.append({"boxes": boxes, "labels": labels})
    return data

# Synchronize prediction and ground truth files
def match_files(preds_dir, targets_dir):
    """
    Match files in preds_dir and targets_dir, keeping only pairs with the same base name.
    """
    preds_files = set(os.listdir(preds_dir))
    targets_files = set(os.listdir(targets_dir))
    
    # Get common files
    common_files = preds_files.intersection(targets_files)
    
    # Generate full paths for common files
    preds_paths = [os.path.join(preds_dir, f) for f in common_files]
    targets_paths = [os.path.join(targets_dir, f) for f in common_files]
    
    return sorted(preds_paths), sorted(targets_paths)

def calculate_iou_and_map(test_labels, pred_path):
    """
    This function calculates the iou and map scores based on predictions and ground truth
    """
    img_width = 640
    img_height = 640

    # Directories containing YOLO label files
    preds_dir = pred_path + "/labels"
    targets_dir = test_labels

    # Match prediction and target files
    preds_files, targets_files = match_files(preds_dir, targets_dir)

    # Prepare prediction and target data
    preds = prepare_data(preds_files, img_width, img_height, include_scores=True)
    targets = prepare_data(targets_files, img_width, img_height, include_scores=False)

    map_metric = MeanAveragePrecision(iou_type="bbox")
    map_metric.update(preds, targets)
    pprint(map_metric.compute())

    iou_metric = IntersectionOverUnion()
    iou_score = iou_metric(preds, targets)
    print(f"Mean IoU Score: {iou_score}")

In [9]:
# Call the function to output IoU and MAP results over the test data
calculate_iou_and_map(test_labels, pred_path)